In [1]:
import os
import sys
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from datetime import datetime
from pathlib import Path
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
from matplotlib.patches import FancyArrowPatch

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [ ]:
colors2 = plt.cm.jet(np.linspace(0.3, 1, 45))
colors = np.vstack((colors2))
mixedmap = LinearSegmentedColormap.from_list('my_colormap', colors)
mixedmap

In [72]:
# %%time
# data_dir = Path('/scratch/wmtsai/temp_mcs/output_stats/MCScount_geomap')
# year_list = np.arange(2001,2021)
# n_times = 0
# for n,year in enumerate(year_list):
#     if n == 0:
#         ds_counts = xr.open_dataset(data_dir / 'MCScounts_geomap_{}.nc'.format(year)).counts
#         n_times += len(list(Path('/neelin2020/mcs_flextrkr/{}'.format(year)).glob('*.nc')))
#     else:
#         tmp = xr.open_dataset(data_dir / 'MCScounts_geomap_{}.nc'.format(year)).counts
#         ds_counts += tmp
#         n_times += len(list(Path('/neelin2020/mcs_flextrkr/{}'.format(year)).glob('*.nc')))

# # save into cause it takes somehow too long...    
# ds_counts.attrs['description'] = 'total counts of hourly MCS mask. 2001-2020'
# ds_counts.to_netcdf('/scratch/wmtsai/temp_mcs/output_stats/MCScount_geomap/MCScount_total_2001.2020.nc')

In [ ]:
# calculate fraction of rainfy times associated with MCSs
year_list = np.arange(2001,2020)

i = 0
n_count = np.zeros((241,1440))
for year in year_list:
    print('processing year: ', year)
    fp_dir = Path('/neelin2020/RGMA_feature_mask/data_product/{}/MERGED_FP'.format(year))
    for month in range(1,13):
        ds_fp = xr.open_dataset(fp_dir / 'GPM_feature_merged_{}_v4_mcsnew.nc'.format(str(month).zfill(2))).sel(latitude=slice(-30,30),longitude=slice(0,359.75))
        prec = ds_fp.precipitationCal
        mcs_tag = ds_fp.mcs_tag
        #prec_amt = prec.where(prec > 1e-3, 0)
        # rainy times
        prec_num = prec.where(prec > 0, 0)
        prec_num = prec_num.where(prec_num == 0, 1)
        # amount of all rainy times
        prec_mcs = prec.where(mcs_tag == 1, 0)
        precamt_mcs = (prec_mcs).sum('time')
        precamt_tot = (prec).sum('time')
        if i == 0:
            frac_pamt = (precamt_mcs/precamt_tot)
            n_map = frac_pamt.where(frac_pamt > 0, 0)
            n_map = n_map.where(n_map == 0, 1)
            n_count += n_map
            i += 1
        else:
            tmp = (precamt_mcs/precamt_tot)
            frac_pamt += tmp
            n_map = frac_pamt.where(frac_pamt > 0, 0)
            n_map = n_map.where(n_map == 0, 1)
            n_count += n_map
            i += 1
            
frac_pamt_avg = frac_pamt/n_count

In [120]:
data_dir = Path('/scratch/wmtsai/temp_mcs/output_stats/MCScount_geomap')
ds_counts = xr.open_dataset(data_dir / 'MCScount_total_2001.2020.nc')
density_counts = ds_counts/n_times*24*365
frac_time = ds_counts.counts/n_times

In [ ]:
# load climatology buoyancy 
stats_dir = Path('/scratch/wmtsai/temp_mcs/output_stats/')
buoy_clim = xr.open_dataset(stats_dir / 'era5_BL_measures_climatology_2002-2014.nc').mean('month')
buoy_tot_clim = buoy_clim.Buoy_TOT.sel(lat=slice(-30,30)) # [K]

fig, ax = plt.subplots(1,1,figsize=(10,4),subplot_kw={'projection': ccrs.PlateCarree(160)})

ax.coastlines(color='brown',linewidth=0.7)
ax.add_feature(cfeat.LAND,zorder=3,edgecolor='brown',facecolor='none',linewidth=0.3) # maskout LAND
ax.add_feature(cfeat.BORDERS,zorder=3,edgecolor='brown',facecolor='none',linewidth=0.3)
ax.add_feature(cfeat.STATES,zorder=3,edgecolor='brown',facecolor='none',linewidth=0.3) 

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='k', alpha=0.4, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 8, 'color': 'k'}
gl.ylabel_style = {'size': 8, 'color': 'k'}
           
# plot seasonal mean of BL_TOT as background
cf = ax.pcolormesh(lon_2deg, lat_2deg, 100*frac_time, vmin=1, vmax=15,
           cmap=mixedmap, transform=ccrs.PlateCarree())
cf.cmap.set_under('w')
cbar = plt.colorbar(cf, ax=ax, shrink=0.35, pad=0.01, extend='both')
cbar.set_label('percentage [%]',fontsize=7)
cbar.set_ticks([1,5,10,15])
cbar.ax.tick_params(labelsize=8)

# BL climatology, flip negative sign for plotting solid contours
cf1 = ax.contour(buoy_tot_clim.lon, buoy_tot_clim.lat, 340*3/9.8*buoy_tot_clim, levels=[-10,-5,0],
          colors=['k'], linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(cf1, fontsize=6)
cf2 = ax.contour(buoy_tot_clim.lon, buoy_tot_clim.lat, 340*3/9.8*buoy_tot_clim, levels=[-10],
          colors=['k'], linewidths=1.5, transform=ccrs.PlateCarree())
ax.clabel(cf2, fontsize=6)

# precipitation fraction explained 
frac_ptime_mask = frac_ptime_avg.where(frac_ptime_avg > 0.5, 0) # greater than 0.5
frac_ptime_mask = frac_ptime_mask.where(frac_ptime_mask == 0, 1)
#ax.contour(test.longitude, test.latitude, test, levels=[0.5,0.6,0.7,0.8],
#             linewidths=1, colors=['m'], transform=ccrs.PlateCarree())
ax.contourf(frac_ptime_mask.longitude, frac_ptime_mask.latitude, frac_ptime_mask, levels=[0.5,1],
            colors='none', hatches=['....'], transform=ccrs.PlateCarree())
                
#ax.set_title('frequency of tropical MCSs, 2001-2020, (counts/total_times)', fontsize=9);
ax.text(x=-170, y=18, s='A.', fontsize=15, fontweight='bold',
        bbox=dict(facecolor='none', alpha=0.7, edgecolor='none', pad=2.5, lw=0.5))
ax.set_ylim([-30,30])
ax.set_xlim([-180,180])

plt.show()
fig.savefig('/scratch/wmtsai/temp_mcs/fig/Fig1a_MCSfrequency_BLclim_PamtFrac.png',dpi=600,
            bbox_inches='tight', transparent=False)

In [ ]:
test = xr.open_dataset('/neelin2020/ERA-5_mse/2001/ERA-5.mse.2001.01.test.nc')
test.mse_sfc.isel(time=0).plot()